In [7]:
import requests
import pandas as pd
from io import StringIO

def fetch_worldbank_gini_data():
    """
    Fetches GINI Index api for our five chosen countries and puts them into a dataframe, Also bounds the GINI INDEX from 0-1

    Returns:
        pd.DataFrame: A Df with columns [country, indicator, indicator_name, date, value]
    """

    # GINI index endpoints for specified countries
    endpoints = {
        "USA_GINI": "https://api.worldbank.org/v2/country/usa/indicator/SI.POV.GINI?format=json&per_page=1000",
        "BRA_GINI": "https://api.worldbank.org/v2/country/br/indicator/SI.POV.GINI?format=json&per_page=1000",
        "GBR_GINI": "https://api.worldbank.org/v2/country/gbr/indicator/SI.POV.GINI?format=json&per_page=1000",
        "ARG_GINI": "https://api.worldbank.org/v2/country/arg/indicator/SI.POV.GINI?format=json&per_page=1000",
        "DEU_GINI": "https://api.worldbank.org/v2/country/deu/indicator/SI.POV.GINI?format=json&per_page=1000",
    }

    all_data = []

    for label, url in endpoints.items():
        response = requests.get(url)
        print(f"{label} status: {response.status_code}")
        data = response.json()

        if isinstance(data, list) and len(data) > 1:
            for entry in data[1]:
                value = entry["value"]
                if value is not None:
                    value = value / 100  # Normalize to 0–1

                all_data.append({
                    "country": entry["country"]["value"],
                    "date": int(entry["date"]),
                    "value": value
                })

    df_all = pd.DataFrame(all_data)
    df_all = df_all.sort_values(by=["country", "date"], ascending=[True, False])

    return df_all

In [8]:
# run the function and form the DF
df = fetch_worldbank_gini_data()
print(df.head())

USA_GINI status: 200
BRA_GINI status: 200
GBR_GINI status: 200
ARG_GINI status: 200
DEU_GINI status: 200
       country  date  value
195  Argentina  2024    NaN
196  Argentina  2023    NaN
197  Argentina  2022  0.407
198  Argentina  2021  0.424
199  Argentina  2020  0.427


In [9]:
# Show the whole dataframe (just to visualize)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


print(df)

            country  date  value
195       Argentina  2024    NaN
196       Argentina  2023    NaN
197       Argentina  2022  0.407
198       Argentina  2021  0.424
199       Argentina  2020  0.427
200       Argentina  2019  0.433
201       Argentina  2018  0.417
202       Argentina  2017  0.414
203       Argentina  2016  0.423
204       Argentina  2015    NaN
205       Argentina  2014  0.418
206       Argentina  2013  0.411
207       Argentina  2012  0.414
208       Argentina  2011  0.427
209       Argentina  2010  0.437
210       Argentina  2009  0.438
211       Argentina  2008  0.450
212       Argentina  2007  0.463
213       Argentina  2006  0.464
214       Argentina  2005  0.478
215       Argentina  2004  0.485
216       Argentina  2003  0.510
217       Argentina  2002  0.538
218       Argentina  2001  0.533
219       Argentina  2000  0.510
220       Argentina  1999  0.498
221       Argentina  1998  0.507
222       Argentina  1997  0.491
223       Argentina  1996  0.495
224       

In [11]:
import plotly.express as px



# Drop the NA values
df_clean = df.dropna(subset=["value"])


fig = px.histogram(
    df_clean,
    x="value",
    color="country",
    nbins=50,
    barmode="overlay", 
    opacity=0.6,
    title="Distribution of GINI Index by Country",
    labels={"value": "GINI Index"}
)

fig.update_layout(
    xaxis_title="GINI Index",
    yaxis_title="Count",
    legend_title="Country"
)

fig.show()

In [ ]:
fig = px.line(
    df_clean,
    x="date",
    y="value",
    color="country",
    markers=True,
    title="GINI Index Over Time by Country",
    labels={"value": "GINI Index", "date": "Year", "country": "Country"}
)

fig.update_layout(
    xaxis=dict(tickmode='linear', dtick=1), 
    yaxis_title="GINI Index",
    legend_title="Country",
    template="plotly_white"
)

fig.show()